In [1]:
import vtk
import os
import numpy as np
import itertools
import math, random

random.seed = 42
import copy

import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
from torchvision import transforms, utils
from torch.utils.data import Subset
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

import scipy.spatial.distance
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
device = "cuda"
import pandas as pd
from sklearn.metrics import confusion_matrix
import seaborn as sns
import pointnetfunct.functions as fun
import pointnetfunct.data_process_ml as data_process_ml

from pointnetfunct.PointNet_dataset import Aneux_Dataset_load
from pointnetfunct.PointNet_trainingfunct import run_model_3multi_head_dnn
from pointnetfunct.PointNet_trainingfunct import run_model_get
from pointnetfunct.PointNet_trainingfunct import load_model
from pointnetfunct.PointNet_struct import Tnet,Transform,PointNet,PointNet_2Multihead,PointNet_3Multihead,PointNet_3Multihead_withDNN,PointNet_2Multimodal_withDNN
from pointnetfunct.evaluation import show_cm_dl,show_graph,print_values,draw_rocgraph
from pointnetfunct.test_functions import pointnet_2branch_testcase,pointnet_3_branch_testcase,pointnet_testcase

#load datasets

morpho_path = ".\AneuX\data-v1.0\data\morpho-per-cut.csv"
patient_path = ".\AneuX\data-v1.0\data\clinical.csv"
morpho_data_patient = data_process_ml.read_and_combine_data(morpho_path,patient_path)
merged_dataset = data_process_ml.encode_column(morpho_data_patient)
merged_dataset = data_process_ml.drop_columns(merged_dataset)
morpho_data_cut1,morpho_data_dome = data_process_ml.output_cut1anddome(merged_dataset)
#load Aneux Dataset
Aneux_1000pt_Dataset_uniform = Aneux_Dataset_load('./Datasets/Aneux_Dataset_1000pt_sample_600train.pt')
Aneux_1000pt_Dataset_uniform_test = Aneux_Dataset_load('./Datasets/Aneux_Dataset_1000pt_sample_100test.pt')

Aneux_2000pt_Dataset_uniform = Aneux_Dataset_load('./Datasets/Aneux_Dataset_2000pt_sample_600train.pt')
Aneux_2000pt_Dataset_uniform_test = Aneux_Dataset_load('./Datasets/Aneux_Dataset_2000pt_sample_100test.pt')

Aneux_1000pt_Dataset_ppd = Aneux_Dataset_load('./Datasets/Aneux_Dataset_1000pt_ppd_600train.pt')
Aneux_1000pt_Dataset_ppd_test = Aneux_Dataset_load('./Datasets/Aneux_Dataset_1000pt_ppd_100test.pt')

Aneux_2000pt_Dataset_ppd = Aneux_Dataset_load('./Datasets/Aneux_Dataset_2000pt_ppd_600train.pt')
Aneux_2000pt_Dataset_ppd_test = Aneux_Dataset_load('./Datasets/Aneux_Dataset_2000pt_ppd_100test.pt')

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


d:\Universityofleeds\MSC_PROG\Msc_project\code\pointnetfunct\data_process_ml.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  morpho_data_cut1.drop(morpho_data_cut1.columns[3:23], axis=1, inplace=True)
d:\Universityofleeds\MSC_PROG\Msc_project\code\pointnetfunct\data_process_ml.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  morpho_data_cut1.drop(['source_x',"cuttype","dataset"], axis=1, inplace=True)
d:\Universityofleeds\MSC_PROG\Msc_project\code\pointnetfunct\data_process_ml.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation

In [7]:
Dataset = Aneux_2000pt_Dataset_uniform_test
Dataset.cuttype = "cut1"
train_size = int(len(Dataset) * 0.8) # 50% training data
valid_size = len(Dataset) - train_size
train_data, valid_data = random_split(Dataset, [train_size, valid_size])
train_loader = torch.utils.data.DataLoader(
    train_data,
    batch_size=20,
    shuffle=True,
    #num_workers=2, 
    pin_memory=True
)

valid_loader = torch.utils.data.DataLoader(
    valid_data,
    batch_size=20, # Forward pass only so batch size can be larger
    shuffle=False,
    #num_workers=2, 
    pin_memory=True
)

In [8]:
from pointnetfunct.PointNet_struct import PointNet_2Multimodal_withDNN
from pointnetfunct.PointNet_trainingfunct import run_model_2multi_head_dnn
pointnet_2mh = PointNet_2Multimodal_withDNN(classes=2)
model_path = "./final_models/2000points/cut1/pointnet_cut1_2branch_uniform_2000pt.pth"
load_model(pointnet_2mh,model_path)

run_model_2multi_head_dnn(train_loader_input = train_loader,
                        vaild_loader_input = valid_loader,
                        nepochs = 100, 
                        modelnet = pointnet_2mh,
                        results_path = "./2branch_cut1_uniform_2000pt_transfer", 
                        filename = "/transfer.pt",
                        model_name="./2branch_cut1_uniform_2000pt_transfer.pth")
pointnet_2mh.to("cpu")

epoch: 0 training loss:  0.800 training accuracy:  48.8%  test loss:  0.716 test accuracy:  50.0%
epoch: 1 training loss:  0.703 training accuracy:  48.8%  test loss:  0.689 test accuracy:  55.0%
epoch: 2 training loss:  0.732 training accuracy:  65.0%  test loss:  0.824 test accuracy:  45.0%
epoch: 3 training loss:  0.618 training accuracy:  67.5%  test loss:  0.738 test accuracy:  50.0%
epoch: 4 training loss:  0.565 training accuracy:  65.0%  test loss:  0.825 test accuracy:  40.0%
epoch: 5 training loss:  0.522 training accuracy:  58.8%  test loss:  0.738 test accuracy:  50.0%
epoch: 6 training loss:  0.541 training accuracy:  56.2%  test loss:  0.763 test accuracy:  50.0%
epoch: 7 training loss:  0.542 training accuracy:  60.0%  test loss:  0.781 test accuracy:  50.0%
epoch: 8 training loss:  0.522 training accuracy:  50.0%  test loss:  0.823 test accuracy:  55.0%
epoch: 9 training loss:  0.539 training accuracy:  57.5%  test loss:  0.859 test accuracy:  55.0%
epoch: 10 training l

PointNet_2Multimodal_withDNN(
  (transform): Transform(
    (input_transform): Tnet(
      (conv1): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
      (conv2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
      (conv3): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
      (fc1): Linear(in_features=1024, out_features=512, bias=True)
      (fc2): Linear(in_features=512, out_features=256, bias=True)
      (fc3): Linear(in_features=256, out_features=9, bias=True)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (feature_transform): Tnet(
      (conv1): Conv1d(64,

In [ ]:
pointnet_2mh.to("cpu")
pointnet_copy = PointNet_2Multimodal_withDNN(classes=2)
model_path = "./2branch_cut1_uniform_2000pt_transfer.pth"
#pointnet_copy = PointNet_2Multimodal_withDNN(classes=2)
# model_path = "./final_models/2000points/cut1/pointnet_cut1_2branch_uniform_2000pt.pth"
load_model(pointnet_copy,model_path)

y_true = []
y_pred = []
y_pred_result = []
y_probs = []
pointnet_copy = pointnet_copy.to("cpu")
with torch.no_grad():
    for inputs_v,inputs_v2,inputs_v3,labels in valid_loader:
        inputs_v = inputs_v.to(torch.float32)
        inputs_v = inputs_v.squeeze(1).permute(0, 2, 1)
        inputs_v2 = inputs_v2.to(torch.float32)
        inputs_v2 = inputs_v2.squeeze(1).permute(0, 2, 1)
        inputs_v3 = inputs_v3.to(torch.float32)
        
        labels = labels.to(torch.long)
        outputs = pointnet_copy.forward(inputs_v,inputs_v3)
        
        #print(torch.exp(outputs))

        outputs_value = (torch.max(torch.exp(outputs), 1)[0]).data.cpu().numpy()
        outputs_result = (torch.max(torch.exp(outputs), 1)[1]).data.cpu().numpy()
        
        
        #print(outputs)
        y_pred.extend(outputs_value)
        y_pred_result.extend(outputs_result)
            
        labels = labels.data.cpu().numpy()
        y_true.extend(labels) 
        y_probs.extend(outputs)

from pointnetfunct.evaluation import show_cm_dl,show_graph,print_values,draw_rocgraph

draw_rocgraph(y_pred_result,y_true, y_probs, name="2000pt_2branch_AOC", interval=0.04)
show_cm_dl(y_true, y_pred,y_pred_result,name="2000pt_2branch_CM")
print_values (y_true, y_pred_result)